In [63]:
!pip install -q pyomo

In [64]:
from pyomo.environ import *

In [65]:
import numpy as np

In [66]:
coef = np.loadtxt('lab4_ex1_coef.txt', delimiter=',')

In [67]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])


(6, 7)
Num rows: 6
Num cols: 7


In [68]:
print(coef)

[[  0.65   0.75  -5.85  -8.35 -12.38  30.     0.  ]
 [ -0.5    0.    -3.8   -0.9    0.9    0.    24.8 ]
 [  1.     0.     0.    -1.     0.    -1.     0.8 ]
 [  0.    -2.9   -1.9    0.     0.    -0.9  -71.4 ]
 [  0.     0.     0.     4.7    8.02   6.15  95.  ]
 [ -0.7   -1.4    7.9   -2.9    0.9    0.   -47.  ]]


In [69]:
model = ConcreteModel()

In [70]:
N = coef.shape[1]-1

In [71]:
M = coef.shape[0]-1

In [72]:
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(6,)
[  0.65   0.75  -5.85  -8.35 -12.38  30.  ]


In [73]:
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(5, 6)
[[-0.5   0.   -3.8  -0.9   0.9   0.  ]
 [ 1.    0.    0.   -1.    0.   -1.  ]
 [ 0.   -2.9  -1.9   0.    0.   -0.9 ]
 [ 0.    0.    0.    4.7   8.02  6.15]
 [-0.7  -1.4   7.9  -2.9   0.9   0.  ]]


In [74]:
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[ 24.8   0.8 -71.4  95.  -47. ]


In [75]:
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[ 24.8   0.8 -71.4  95.  -47. ]


In [76]:
lower_bound = 0
upper_bound = np.inf

In [77]:
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5]


In [78]:
model.x = Var(col_indices)

In [79]:
row_indices = np.arange(M)
print(row_indices)

[0 1 2 3 4]


In [80]:
model.constraints = ConstraintList()

In [81]:
for i in row_indices:
  model.constraints.add(summation(constr_coef[i],model.x) <= constr_rhs[i])

In [82]:
for j in col_indices:
  model.x[j].setlb(lower_bound)
  model.x[j].setub(upper_bound)

In [83]:
model.objective = Objective(expr = summation(obj_coef,model.x), sense=maximize)

In [84]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [85]:
!apt-get install -y -qq coinor-cbc

In [86]:
opt_cbc = SolverFactory('cbc')

In [87]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


# ***Exercise 1,Answer(7):*** The solver termination condition is showing unbounded and solver status showing warning message because the constraints and bounds of constraints that are chosen is not right to give the optimal solution so the termination condition is unbounded.

In [88]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()

ERROR:pyomo.core:evaluating object as numeric value: x[0]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object x[0]


ValueError: ignored

In [92]:
model.x.setub(21)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          0 :     0 : 16.726154 :    21 : False : False :  Reals
          1 :     0 :      21.0 :    21 : False : False :  Reals
          2 :     0 :       0.0 :    21 : False : False :  Reals
          3 :     0 :  2.031618 :    21 : False : False :  Reals
          4 :     0 :       0.0 :    21 : False : False :  Reals
          5 :     0 : 13.894536 :    21 : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65

In [93]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [94]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  426.49406980000003

Decision Variables
x[ 0 ] =  16.726154
x[ 1 ] =  21.0
x[ 2 ] =  0.0
x[ 3 ] =  2.031618
x[ 4 ] =  0.0
x[ 5 ] =  13.894536

Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :  None :        -10.1915332 :  24.8
      2 :  None : 0.8000000000000007 :   0.8
      3 :  None :        -73.4050824 : -71.4
      4 :  None :  95.00000100000001 :  95.0
      5 :  None :              -47.0 : -47.0


# ***Exercise 2,Answer (9):*** I have the changed the upper bound of all the variables as 21, and then i get the optimal solution and I got the optimal function value is 426.49406980000003 and the value of variables is 

x[ 0 ] =  16.726154

x[ 1 ] =  21.0

x[ 2 ] =  0.0

x[ 3 ] =  2.031618

x[ 4 ] =  0.0

x[ 5 ] =  13.894536


And activity of all the five constraints is active because in solution it is equal to RHS.
